In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 39.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import requests
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
import concurrent.futures
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


get the data frames calling the api

In [ ]:
def get_data():
    url_players = "https://fantasy.premierleague.com/api/bootstrap-static/"

    response = requests.get(url_players)

    if response.status_code == 200:
        fixtures_data = response.json()
        # do something with the fixtures data here
    else:
        print(f"Failed to retrieve fixtures data: {response.status_code}")
        exit()
    return fixtures_data


create the players and teams dataframe from the response

In [ ]:
def get_players_data(fixtures_data):
    df_players = pd.DataFrame(fixtures_data['elements'])
    return df_players

In [ ]:
def get_teams_data(fixtures_data):
    df_teams = pd.DataFrame(fixtures_data['teams'])
    return df_teams

create a merged df, each player will have its team data

In [ ]:
def merge_players_teams(df_players, df_teams):

    merged_df = pd.merge(df_players, df_teams, left_on='team',right_on='id', suffixes=('_player', '_team'), how='inner')

    merged_df1 = merged_df[merged_df['chance_of_playing_next_round'] >50.0]
    merged_df2 = merged_df[pd.isna(merged_df['chance_of_playing_next_round'])]

    merged_df = pd.concat([merged_df1, merged_df2], ignore_index=True)

    merged_df = merged_df[['id_player','element_type','team','web_name']]

    return merged_df

create the fixtures data frame calling the api

In [ ]:
def get_gw_data(dt):

    df = pd.DataFrame(dt['events'])
    df_teams = pd.DataFrame(dt['teams'])
    next_updated_gw = df[df['is_next'] == True]
    num_updated_next_gw = next_updated_gw.index[0] + 1


    url_events = "https://fantasy.premierleague.com/api/fixtures/?event="+str(num_updated_next_gw)
    #print(url_events)
    #print(num_updated_next_gw)

    response = requests.get(url_events)

    if response.status_code == 200:
        fixtures_data = response.json()
        # do something with the fixtures data here
    else:
        print(f"Failed to retrieve fixtures data: {response.status_code}")
        exit()

    df_fix = pd.DataFrame(fixtures_data)
    return df_fix

create the players history in the current season, using the api

In [ ]:
def get_players_hist(merged_df):

    id_game = list()
    id_player = list()
    id_rival = list()
    team = list()
    was_home = list()
    fpl_total_points = list()
    player_pos = list()
    minu = list()

    for i in merged_df['id_player']:
        player_hist_url = "https://fantasy.premierleague.com/api/element-summary/"+str(i)+"/"
        response = requests.get(player_hist_url)

        # Check if the request was successful (HTTP status code 200 means success)
        if response.status_code == 200:
            # Extract the player's history from the JSON response
            history = response.json()

        else:
            # If the request was not successful, print the error message
            print(f"Error: {response.text}")
            exit()

        df_player_hist = pd.DataFrame(history['history'])


        if not df_player_hist.columns.tolist():
            continue

        df_player_hist = df_player_hist.drop(['kickoff_time', 'value', 'transfers_balance', 'selected'  ,'transfers_in', 'transfers_out'],axis= 1)


        team_id = merged_df.loc[merged_df['id_player'] == i]['team']
        team_id = int(team_id.iloc[0])

        player_type = merged_df.loc[merged_df['id_player'] == i]['element_type'].iloc[0]
        l = len(df_player_hist)
        for j in range(l):
            #x = df_player_hist.iloc[j]['fixture']
            #id_game.append(x)
            id_player.append(df_player_hist.iloc[j]['element'])
            id_rival.append(df_player_hist.iloc[j]['opponent_team'])
            team.append(team_id)
            was_home.append(df_player_hist.iloc[j]['was_home'])
            fpl_total_points.append(df_player_hist.iloc[j]['total_points'])
            player_pos.append(player_type)
            minu.append(df_player_hist.iloc[j]['minutes'])


    players_games = pd.DataFrame({'id_player':id_player, 'id_rival': id_rival, 'player_team_id':team,
                                  'was_home':was_home, 'minutes': minu , 'fpl_total_points': fpl_total_points, 'player_pos':player_pos})

    return players_games.sort_values('fpl_total_points', ascending= False)

alternative to get hist:

In [ ]:
def get_players_hist2(merged_df):
    player_ids = merged_df['id_player'].tolist()
    player_hist_urls = [f"https://fantasy.premierleague.com/api/element-summary/{i}/" for i in player_ids]

    player_data = []
    for url, player_id in zip(player_hist_urls, player_ids):
        response = requests.get(url)
        if response.status_code == 200:
            history = response.json()
            if 'history' in history:
                player_data.extend(history['history'])

    if not player_data:
        print("No player history data found.")
        return None

    df_player_hist = pd.DataFrame(player_data)
    if df_player_hist.empty:
        print("Player history DataFrame is empty.")
        return None

    # Assuming df_teams contains team information

    # Merge with team strength data
    df_player_hist = pd.merge(df_player_hist, df_teams[['id', 'strength']], left_on='opponent_team', right_on='id')

    # Drop unnecessary columns
    df_player_hist = df_player_hist.drop(['kickoff_time', 'value', 'transfers_balance', 'selected', 'transfers_in', 'transfers_out'], axis=1)

    player_game_data = []

    for _, row in merged_df.iterrows():
        player_id = row['id_player']
        team_id = row['team']
        player_type = row['element_type']
        team_str = df_teams[df_teams['id'] == team_id]['strength'].iloc[0]

        player_hist_subset = df_player_hist[df_player_hist['element'] == player_id]

        if not player_hist_subset.empty:
            for _, game_row in player_hist_subset.iterrows():
                player_game_data.append({
                    'id_player': player_id,
                    'id_rival': game_row['opponent_team'],
                    'player_team_id': team_id,
                    'was_home': game_row['was_home'],
                    'minutes': game_row['minutes'],
                    'fpl_total_points': game_row['total_points'],
                    'player_pos': player_type
                })

    players_games = pd.DataFrame(player_game_data)

    return players_games.sort_values('fpl_total_points', ascending=False)

create a data frame with the data about the upcoming fixture, with column to predict

In [ ]:
def get_results_to_predict(df_fix, merged_df):
    #id_game = list() #
    id_player = list() #
    id_rival = list() #
    team = list() # i
    is_home = list() #
    fpl_total_points = list() # to predict
    player_pos = list() #
    minu = list() #
    print("@@@@@")
    print(df_gw.head(10))
    print(type(df_gw['team_a']))
    player_team_id = list()
    for i in df_gw['team_a']:
        x = 2
        print("i is "+ str(i))
        id_rival_x = df_gw.loc[df_gw['team_a'] == i]['team_h'].iloc[0]
        #print("team is " + str(i) + " rival is " + str(id_rival_x))
        #print("hello")

        temp_df = df_merged.loc[df_merged['team'] == i]
        temp_ser = temp_df['id_player']
        temp_lst = temp_ser.to_list()
        id_player.extend(temp_lst)
        lenp = len(temp_lst)
        #print(lenp)

        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = id_rival_x
        id_rival.extend(temp_lst2)

        print(len(id_rival))

        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = False
        is_home.extend(temp_lst2)

        id_game_x = df_gw.loc[df_gw['team_a'] == i]['id'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = id_game_x
        #id_game.extend(temp_lst2)


        team_strength_x = df_gw.loc[df_gw['team_a'] == i]['team_a_difficulty'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = team_strength_x
        #team_strength.extend(temp_lst2)
        print("team is " + str(i) + " strength is " + str(team_strength_x))

        rival_strength_x = df_gw.loc[df_gw['team_a'] == i]['team_h_difficulty'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = rival_strength_x
        #rival_strength.extend(temp_lst2)

        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = i
        player_team_id.extend(temp_lst2)


        for j in temp_ser.tolist():
            posi_x = df_merged.loc[df_merged['id_player'] == j]['element_type'].iloc[0]
            player_pos.append(posi_x)



    for i in df_gw['team_h']:
        x = 2

        id_rival_x = df_gw.loc[df_gw['team_h'] == i]['team_a'].iloc[0]


        temp_df = df_merged.loc[df_merged['team'] == i]
        temp_ser = temp_df['id_player']
        temp_lst = temp_ser.to_list()
        id_player.extend(temp_lst)
        lenp = len(temp_lst)
        #print(lenp)


        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = id_rival_x
        id_rival.extend(temp_lst2)

        #print(len(id_rival))

        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = True
        is_home.extend(temp_lst2)

        id_game_x = df_gw.loc[df_gw['team_h'] == i]['id'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = id_game_x
        #id_game.extend(temp_lst2)


        team_strength_x = df_gw.loc[df_gw['team_h'] == i]['team_h_difficulty'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = team_strength_x
        #team_strength.extend(temp_lst2)

        rival_strength_x = df_gw.loc[df_gw['team_h'] == i]['team_a_difficulty'].iloc[0]
        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = rival_strength_x
        #rival_strength.extend(temp_lst2)

        temp_lst2 = np.empty(lenp)
        temp_lst2[:] = i
        player_team_id.extend(temp_lst2)


        for j in temp_ser.tolist():

            posi_x = df_merged.loc[df_merged['id_player'] == j]['element_type'].iloc[0]
            player_pos.append(posi_x)


    minu = np.empty(len(id_player))
    minu[:] = 90


    fpl_total_points = np.empty(len(id_player))
    fpl_total_points[:] = np.nan


    print(len(id_player))
    print(len(id_rival))
    print(len(player_team_id))
    print(len(is_home))
    print(len(minu))
    print(len(fpl_total_points))
    print(len(fpl_total_points))
    print(len(player_pos))

    dicti = pd.DataFrame({'id_player':id_player, 'id_rival': id_rival, 'player_team_id':player_team_id,
                                  'is_home':is_home, 'minutes': minu , 'fpl_total_points': fpl_total_points, 'player_pos':player_pos})
    #for i in dicti.values():
    #  print(len(i))

    print("%%%%%")

    players_upcoming_games = pd.DataFrame(dicti)
    print(players_upcoming_games.head())
    return players_upcoming_games

main

In [ ]:
dt = get_data()
df_gw = get_gw_data(dt)
df_players = get_players_data(dt)


print(df_players[df_players['web_name'] == 'Haaland']['id'].head())
df_teams = get_teams_data(dt)
df_merged = merge_players_teams(df_players,df_teams)
print(df_merged[df_merged['id_player'] == 709]['web_name'].head())


#print(df_merged.columns.tolist())

df_merged = merge_players_teams(df_players, df_teams)

df_hist = get_players_hist2(df_merged)
#df_hist = get_h2(df_merged)
#print(df_hist.sort_values(by='fpl_total_points', ascending=False))
df_guess = get_results_to_predict(df_gw, df_merged)

print(df_guess.head())

df_guess.rename(columns={'is_home': 'was_home'}, inplace=True)
df_guess = df_guess[df_hist.columns.to_list()]
print(df_guess.head())


385    351
Name: id, dtype: int64


NameError: name 'get_teams_data' is not defined

In [ ]:
#print(df_hist.head())
#print(df_merged[df_merged['id_player'] == 26].head())
#print(df_hist.head())
print(df_hist[df_hist['id_player'] == 355].head())
#print(df_guess[df_guess['player_team_id'] == 1])

      id_player  id_rival  player_team_id  was_home  minutes  \
7699        355        20              13      True       82   
7678        355        10              13      True       90   
7688        355         7              13     False       90   
7680        355        14              13     False       90   
7694        355         6              13     False       79   

      fpl_total_points  player_pos  
7699                21           4  
7678                20           4  
7688                16           4  
7680                16           4  
7694                13           4  


guessing the ucoming fpl score of every player

In [ ]:
guess_tmp = df_guess.copy()
hist_tmp = df_hist.copy()
hist_tmp = hist_tmp.drop('minutes', axis=1)
guess_tmp = guess_tmp.drop('minutes', axis=1)

# Encode categorical features using Label Encoding
label_encoders = {}
categorical_cols = df_hist.columns.to_list()
categorical_cols.remove('fpl_total_points')
categorical_cols.remove('minutes')
print(categorical_cols)

for col in categorical_cols:
    le = LabelEncoder()
    hist_tmp[col] = le.fit_transform(hist_tmp[col])
    label_encoders[col] = le

# Split the dataset into features (X) and target (Y)
X = hist_tmp.drop('fpl_total_points', axis=1)
Y = hist_tmp['fpl_total_points']

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build and train a Random Forest Regressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = model.predict(X_test)

mse = mean_squared_error(Y_test, Y_pred)
print(f'Mean Squared Error: {mse}')

categorical_cols = df_guess.columns.to_list()
categorical_cols.remove('fpl_total_points')
categorical_cols.remove('minutes')
print(categorical_cols)


# Encode categorical features in the new dataset using new Label Encoders
new_label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    guess_tmp[col] = le.fit_transform(guess_tmp[col])
    new_label_encoders[col] = le

# Use the trained model to make predictions on the new dataset
new_predictions = model.predict(guess_tmp.drop('fpl_total_points', axis=1))

# Add the predictions as a new column 'Predicted_Income' in the new dataset
df_guess['fpl_total_points'] = new_predictions

# Display the new dataset with predictions

#print("dfguess after predicting")
#print(df_guess[df_guess['player_team_id'] == 2])
df_guess.rename(columns={'was_home': 'is_home'}, inplace=True)
sorted_df = df_guess.sort_values(by='fpl_total_points', ascending=False)
print(sorted_df)


['id_player', 'id_rival', 'player_team_id', 'was_home', 'player_pos']
Mean Squared Error: 6.565445998093295
['id_player', 'id_rival', 'player_team_id', 'was_home', 'player_pos']
     id_player  id_rival  player_team_id  is_home  minutes  fpl_total_points  \
277         29       9.0             1.0      1.0     90.0             11.39   
276         26       9.0             1.0      1.0     90.0              9.28   
268         12       9.0             1.0      1.0     90.0              8.30   
264          5       9.0             1.0      1.0     90.0              8.22   
448        308      20.0            11.0      1.0     90.0              7.98   
..         ...       ...             ...      ...      ...               ...   
519        471      18.0            17.0      1.0     90.0             -0.15   
159        548      11.0            20.0      0.0     90.0             -0.16   
523        483      18.0            17.0      1.0     90.0             -0.17   
7          257       1

runnings samples and outputs.


In [ ]:
sorted_df = pd.merge(sorted_df, df_merged[['id_player', 'web_name']], on='id_player', how='inner')
print(sorted_df.head(10))


print(sorted_df[sorted_df['player_pos'] == 1])


   id_player  id_rival  player_team_id  is_home  minutes  fpl_total_points  \
0         29       9.0             1.0      1.0     90.0             11.39   
1         26       9.0             1.0      1.0     90.0              9.28   
2         12       9.0             1.0      1.0     90.0              8.30   
3          5       9.0             1.0      1.0     90.0              8.22   
4        308      20.0            11.0      1.0     90.0              7.98   
5        775      18.0            17.0      1.0     90.0              7.97   
6         77       7.0             3.0      0.0     90.0              7.79   
7        353      19.0            13.0      1.0     90.0              7.28   
8        172      16.0             6.0      1.0     90.0              7.10   
9        526      13.0            19.0      0.0     90.0              6.59   

   player_pos       web_name  
0           2          White  
1           3       Trossard  
2           3     Martinelli  
3           2    